In [1]:
import cv2
import numpy as np
from utils import stackImages,reorder      # in my local pc

In [2]:
img=cv2.imread("test.jpg")
widthImg=img.shape[1]
heightImg=img.shape[0]

In [3]:
img_blank=np.zeros_like(img)
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_threshold=cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,15,10)
# img_blur=cv2.GaussianBlur(gray,(7,7),0)
# img_canny=cv2.Canny(img_blur,15,15)            # Canny edge detector
# kernal=np.ones((5,5),dtype=np.uint8)                           ## thicknned the edge by increase iteration using dialation
# img_dialated=cv2.dilate(img_canny,kernal,iterations=2)
# img_erode=cv2.erode(img_dialated,kernal,iterations=2)             ## thinned the edge by increase iteration using erodation

In [4]:
img_contour=img.copy()

def get_contour(img):
    biggest=np.array([])
    maxArea=0
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
    for cnt in contours:
        area=cv2.contourArea(cnt)
        if (area>5000):
            cv2.drawContours(img_contour,cnt,-1,(255,0,0),5)
            peri=cv2.arcLength(cnt,True)
            approx=cv2.approxPolyDP(cnt,0.02*peri,True)      ## return coordinates of points in contour 
            if area>maxArea and len(approx)==4:
                biggest=approx
                maxArea=area
            x,y,w,h=cv2.boundingRect(approx)                 ## bound retangle accros contour or object 
            #cv2.rectangle(img_contour,(x,y),(w,h),(255,0,255),4)
        cv2.drawContours(img_contour,cnt,-1,(255,0,0),5)
    cv2.drawContours(img_contour,biggest,-1,(255,0,255),30)
    return biggest

biggest=get_contour(img_threshold)       ## may also use img_erode to get contours

In [5]:
if biggest.size!=0:
    biggest=reorder(biggest)
    try:
        pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
        pts2 =  np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
        matrix = cv2.getPerspectiveTransform(pts1, pts2)  # GER
        imgWarpColored = img.copy()
        imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    except:
        print("Images are of Bad quality......Show another image......")
        imgWarpColored = img.copy()
imgWarpColored=imgWarpColored[5:imgWarpColored.shape[0]-5,5:imgWarpColored.shape[1]-5]
imgWarpColored=cv2.resize(imgWarpColored,(heightImg,widthImg))

In [6]:
img_stack=stackImages(0.5,([img,img_threshold,imgWarpColored]))
cv2.imshow('Image',img_stack)
cv2.waitKey(0)
cv2.destroyAllWindows()